In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs
from reversible2.invert import invert

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs

In [ ]:
rng = RandomState(201904113)#2 ganz gut # 13 sehr gut)

X = rng.randn(5,2) * np.array([1,0])[None] + np.array([-1,0])[None]


plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1])
plt.scatter([-1],[0], color='black')
plt.xlim(-2.5,5.5)
plt.ylim(-4,4)

In [ ]:
plt.scatter(X[:2,0], X[:2,1])
plt.scatter(X[3:5,0], X[3:5,1])

In [ ]:
train_inputs = np_to_var(np.concatenate((X[:2],X[3:5])), dtype=np.float32)
cuda = False

In [ ]:
from reversible2.distribution import TwoClassDist

from reversible2.blocks import dense_add_block, conv_add_block_3x3
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter


set_random_seeds(2019011641, cuda)
feature_model_a = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)
if cuda:
    feature_model_a.cuda()
from reversible2.ot_exact import ot_euclidean_loss_for_samples
class_dist = TwoClassDist(2,0, [0,1])
if cuda:
    class_dist.cuda()

optim_model_a = th.optim.Adam(feature_model_a.parameters())
optim_dist = th.optim.Adam(class_dist.parameters(), lr=1e-3)

In [ ]:
th.randn(outs_a.shape)

In [ ]:
perturb_norm

In [ ]:
from reversible.gaussian import get_gauss_samples
n_epochs = 3001
for i_epoch in range(n_epochs):
    outs_a = feature_model_a(train_inputs)
    
    perturbations = th.randn(outs_a.shape) * 0.01
    outs_perturbed = outs_a + perturbations
    inverted_perturbed = invert(feature_model_a, outs_perturbed)
    out_dists = th.norm(perturbations, p=2, dim=1)
    in_dists = th.norm(train_inputs - inverted_perturbed, p=2, dim=1)
    lip_loss = F.relu((in_dists - out_dists) ** 2)
    log_probs = class_dist.get_class_log_prob(0, outs_a)
    nll_loss = -th.mean(log_probs)
    loss = nll_loss + lip_loss
    optim_model_a.zero_grad()
    loss.backward()
    optim_model_a.step()
    if i_epoch % (n_epochs // 20) == 0:
        fig = plt.figure(figsize=(6,6))
        model = feature_model_a
        rng = RandomState(201904114)
        outs = model(train_inputs)
        other_X = rng.randn(100,2) * np.array([1,0])[None] + np.array([-1,0])[None]
        other_X = other_X[np.argsort(other_X[:,0])]
        other_ins = np_to_var(other_X, dtype=np.float32)
        other_outs = model(other_ins)

        plt.plot(var_to_np(other_outs[:,0]), var_to_np(other_outs[:,1]), label="All Outputs")
        plt.scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30, c=[seaborn.color_palette()[2]],
                   label="Actual data outputs")
        plt.legend()
        display_close(fig)


In [ ]:
(in_dists - out_dists)

In [ ]:
class_dist.get_mean_std(0)

In [ ]:
from reversible2.distribution import TwoClassDist

from reversible2.blocks import dense_add_block, conv_add_block_3x3
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter


set_random_seeds(2019011641, cuda)
feature_model_a = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)
if cuda:
    feature_model_a.cuda()
from reversible2.ot_exact import ot_euclidean_loss_for_samples
class_dist = TwoClassDist(2,0, [0,1])
if cuda:
    class_dist.cuda()

optim_model_a = th.optim.Adam(feature_model_a.parameters())
optim_dist = th.optim.Adam(class_dist.parameters(), lr=1e-2)

In [ ]:
from reversible.gaussian import get_gauss_samples
n_epochs = 2001
for i_epoch in range(n_epochs):
    outs_a = feature_model_a(train_inputs)
    perturbations = th.randn(outs_a.shape) * 0.01
    outs_perturbed = outs_a + perturbations
    inverted_perturbed = invert(feature_model_a, outs_perturbed)
    out_dists = th.norm(perturbations, p=2, dim=1)
    in_dists = th.norm(train_inputs - inverted_perturbed, p=2, dim=1)
    eps=1e-8
    lip_loss = th.mean(F.relu(((in_dists - out_dists) / (out_dists +eps)) ** 2))
    log_probs = class_dist.get_class_log_prob(0, outs_a)
    nll_loss = -th.mean(log_probs)
    loss = nll_loss + lip_loss * 100
    optim_dist.zero_grad()
    optim_model_a.zero_grad()
    loss.backward()
    optim_model_a.step()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        display_text("NLL: {:.1E}, Lip: {:.1E}".format(nll_loss.item(), lip_loss.item()))

        fig,axes = plt.subplots(1,2, figsize=(10,4))
        model = feature_model_a
        rng = RandomState(201904114)
        outs = model(train_inputs)
        other_X = rng.randn(100,2) * np.array([1,0])[None] + np.array([-1,0])[None]
        other_X = other_X[np.argsort(other_X[:,0])]
        other_ins = np_to_var(other_X, dtype=np.float32)
        other_outs = model(other_ins)

        axes[0].plot(var_to_np(other_outs[:,0]), var_to_np(other_outs[:,1]), label="All Outputs",
                color=seaborn.color_palette()[1])
        axes[0].scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30, c=[seaborn.color_palette()[0]],
                   label="Actual data outputs")
        axes[0].axis('equal')
        axes[0].set_title("Output space")
        plt.axis('equal')
        samples = class_dist.get_samples(0, 100)
        inverted = invert(feature_model_a, samples)
        axes[1].scatter(var_to_np(inverted)[:,0], var_to_np(inverted)[:,1], s=30, label="Fake/Unknown Samples",
                   c=[seaborn.color_palette()[1]])
        axes[1].scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], s=30, label="Real data",
                   c=[seaborn.color_palette()[0]])
        axes[1].legend(bbox_to_anchor=(1,1,0,0))
        axes[1].set_title("Input space")
        axes[1].axis('equal')
        display_close(fig)


In [ ]:

fig,axes = plt.subplots(1,2, figsize=(10,4))
model = feature_model_a
rng = RandomState(201904114)
outs = model(train_inputs)
other_X = rng.randn(100,2) * np.array([1,0])[None] + np.array([-1,0])[None]
other_X = other_X[np.argsort(other_X[:,0])]
other_ins = np_to_var(other_X, dtype=np.float32)
other_outs = model(other_ins)

axes[0].plot(var_to_np(other_outs[:,0]), var_to_np(other_outs[:,1]), label="All Outputs",
        color=seaborn.color_palette()[1])
axes[0].scatter(var_to_np(outs[:,0]), var_to_np(outs[:,1]), s=30, c=[seaborn.color_palette()[0]],
           label="Actual data outputs")
#axes[0].legend(bbox_to_anchor=(1,1,0,0))
axes[0].set_title("Output space")
axes[0].axis('equal')
samples = class_dist.get_samples(0, 100)
inverted = invert(feature_model_a, samples)
axes[1].scatter(var_to_np(inverted)[:,0], var_to_np(inverted)[:,1], s=30, label="Fake/Unknown Samples",
           c=[seaborn.color_palette()[1]])
axes[1].scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], s=30, label="Real data",
           c=[seaborn.color_palette()[0]])
axes[1].legend(bbox_to_anchor=(1,1,0,0))
axes[1].set_title("Input space")
axes[1].axis('equal')
display_close(fig)

In [ ]:
class_dist.get_mean_std(0)

In [ ]:
nll_loss